In [81]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
# jupyter nbconvert --to html SSE/BE918_SustainabilityChallenges/plastics_per_country.ipynb 

font_style = dict(family="Times New Roman", size=16, color="snow")
transparent_layout = dict(
    paper_bgcolor="rgba(0,0,0,0)",  # transparent background outside plot
    plot_bgcolor="rgba(0,0,0,0)",   # transparent plot area
    font=font_style
)


In [8]:
# --- constants --------------------------------------------------------------
# Plastic-to-volume conversion assumptions
BOTTLE_WEIGHT_G = 8.3         # average 500 mL PET bottle weight
BOTTLE_SIZE_L = 0.5           # bottle volume in liters
LITERS_PER_TONNE_PET = (1_000_000 / BOTTLE_WEIGHT_G) * BOTTLE_SIZE_L  # ~60,241 L/tonne PET

# Climate change impact factors (kg CO2e/L)
PET_CO2_PER_L = 0.2826
CARTON_CO2_PER_L = 0.05435
DELTA_CO2_PER_L = PET_CO2_PER_L - CARTON_CO2_PER_L  # savings per liter switched


In [9]:
# --- input data -------------------------------------------------------------
data = {
    "Company": ["Coca-Cola", "PepsiCo", "Nestlé", "Danone"],
    "Plastic_Usage_tonnes": [2_940_000, 2_100_000, 872_900, 670_107],  # tonnes from reports
}

df = pd.DataFrame(data)
df = df.sort_values(by='Plastic_Usage_tonnes', ascending=False)
df

,Company,Plastic_Usage_tonnes
0,Coca-Cola,2940000
1,PepsiCo,2100000
2,Nestlé,872900
3,Danone,670107


In [10]:
# --- calculations -----------------------------------------------------------
# Convert tonnes of PET to estimated liters of bottled water
df["Estimated_Liters"] = df["Plastic_Usage_tonnes"] * LITERS_PER_TONNE_PET

# Compute CO2 emissions for PET and carton packaging
df["CO2_PET_tonnes"] = df["Estimated_Liters"] * PET_CO2_PER_L / 1000
df["CO2_Carton_tonnes"] = df["Estimated_Liters"] * CARTON_CO2_PER_L / 1000
df["CO2_Saved_tonnes"] = df["CO2_PET_tonnes"] - df["CO2_Carton_tonnes"]

df

,Company,Plastic_Usage_tonnes,Estimated_Liters,CO2_PET_tonnes,CO2_Carton_tonnes,CO2_Saved_tonnes
0,Coca-Cola,2940000,1.771084e+11,5.005084e+07,9.625843e+06,40425000.00
1,PepsiCo,2100000,1.265060e+11,3.575060e+07,6.875602e+06,28875000.00
2,Nestlé,872900,5.258434e+10,1.486033e+07,2.857959e+06,12002375.00
3,Danone,670107,4.036789e+10,1.140797e+07,2.193995e+06,9213971.25


In [76]:
fig1 = go.Figure()

# Transparent bars (just placeholders for proper layout)
fig1.add_trace(go.Bar(
    x=df["Company"],
    y=df["CO2_PET_tonnes"] / 1e6,
    name="PET Bottles",
    marker_color="rgba(0,0,0,0)"  # transparent
))
fig1.add_trace(go.Bar(
    x=df["Company"],
    y=df["CO2_Carton_tonnes"] / 1e6,
    name="Cartons",
    marker_color="rgba(0,0,0,0)"  # transparent
))

# --- overlay bottle and carton images --------------------------------------
# horizontal shifts for grouped bar alignment
x_shift_bottle = -0.18
x_shift_carton = +0.18

for i, row in df.iterrows():
    y_pet = row["CO2_PET_tonnes"] / 1e6
    y_carton = row["CO2_Carton_tonnes"] / 1e6

    # PET bottle images
    fig1.add_layout_image(
        dict(
            source="bottle.png",
            xref="x",
            yref="y",
            x=i + x_shift_bottle,
            y=y_pet,
            sizex=1.3,
            sizey=y_pet,
            xanchor="center",
            yanchor="top",
            layer="below"
        )
    )

    # Carton images
    fig1.add_layout_image(
        dict(
            source="tetraPak.png",
            xref="x",
            yref="y",
            x=i + x_shift_carton,
            y=y_carton,
            sizex=10,
            sizey=y_carton,
            xanchor="center",
            yanchor="top",
            layer="below"
        )
    )

# --- finalize layout --------------------------------------------------------
fig1.update_layout(
    title="Estimated CO₂ Emissions – PET vs. Carton Packaging",
    xaxis_title="Company",
    yaxis_title="Yearly CO₂ Emissions (million tonnes)",
    barmode="group",
    template="plotly_white",
    showlegend=False,  
    **transparent_layout
)

#fig1.update_xaxes(showgrid=False)
fig1.update_yaxes(showgrid=False)


fig1.show()

In [79]:
fig2 = px.bar(
    df,
    x="Company",
    y=df["CO2_Saved_tonnes"] / 1e6,
    color="Company",
    title="Potential Yearly CO₂ Savings if PET Bottles Replaced by Cartons",
    labels={"y": "CO₂ saved (million tonnes)"}
)

fig2.update_traces(marker_color="rgba(0,0,0,0)")
fig2.update_layout(showlegend=False, template="plotly_white", **transparent_layout)
y_val = df.loc[df["Company"] == "Coca-Cola", "CO2_Saved_tonnes"].iloc[0] / 1e6

fig2.add_layout_image(
    dict(
        source="cola.png",     
        xref="x",
        yref="y",
        x="Coca-Cola",        
        y=y_val,             
        sizex=3,       
        sizey=y_val * 1.1,     
        xanchor="center",
        yanchor="top",
        layer="above"
    )
)

y_val = df.loc[df["Company"] == "Danone", "CO2_Saved_tonnes"].iloc[0] / 1e6

fig2.add_layout_image(
    dict(
        source="danone.png",     
        xref="x",
        yref="y",
        x="Danone",        
        y=y_val,             
        sizex=3,       
        sizey=y_val * 1.1,     
        xanchor="center",
        yanchor="top",
        layer="above"
    )
)

y_val = df.loc[df["Company"] == "PepsiCo", "CO2_Saved_tonnes"].iloc[0] / 1e6

fig2.add_layout_image(
    dict(
        source="pepsi.png",     
        xref="x",
        yref="y",
        x="PepsiCo",        
        y=y_val,             
        sizex=3,       
        sizey=y_val * 1.1,     
        xanchor="center",
        yanchor="top",
        layer="above"
    )
)

y_val = df.loc[df["Company"] == "Nestlé", "CO2_Saved_tonnes"].iloc[0] / 1e6

fig2.add_layout_image(
    dict(
        source="nestle.png",     
        xref="x",
        yref="y",
        x="Nestlé",        
        y=y_val,             
        sizex=3,       
        sizey=y_val * 1.1,     
        xanchor="center",
        yanchor="top",
        layer="above"
    )
)

#fig2.update_xaxes(showgrid=False)
fig2.update_yaxes(showgrid=False)

fig2.show()
